In [1]:
import hail as hl
hl.init(default_reference='GRCh38')

Running on Apache Spark version 3.1.1
SparkUI available at http://dataproc-vlad-m.australia-southeast1-a.c.tob-wgs.internal:36165
Welcome to
     __  __     <>__
    / /_/ /__  __/ /
   / __  / _ `/ / /
  /_/ /_/\_,_/_/_/   version 0.2.73-2ab84582d24d
LOGGING: writing to /home/hail/hail-20210923-0916-0.2.73-2ab84582d24d.log


In [4]:
import hail as hl
import numpy as np
from ipywidgets import interact
import math
import pandas as pd
from collections import OrderedDict
import json
from os.path import join

import bokeh
from bokeh.layouts import gridplot, row, widgetbox
from bokeh.plotting import figure, show, output_file
from bokeh.io import output_notebook, push_notebook, export_png
from bokeh.models.widgets import Tabs, Panel
from bokeh.palettes import *
from bokeh.models import *
from typing import *
from bokeh.plotting.helpers import stack
from bokeh.transform import factor_cmap

from gnomad.variant_qc.evaluation import add_rank
from joint_calling import utils

overwrite = True
work_bucket = 'gs://cpg-tob-wgs-test-analysis/jupyter/vsavelyev'
TRUTH_SAMPLE = 'NA12878'

from bokeh.io import show, output_notebook, reset_output
from bokeh.layouts import gridplot
output_notebook()

gvcf_by_key = {
    'from_broad_gvcf': 'gs://cpg-thousand-genomes-test/gvcf/NA12878-from-broad-gvcf.g.vcf.gz',
    'from_cram'      : 'gs://cpg-thousand-genomes-test/gvcf/NA12878-from-cram.g.vcf.gz',
    'realign'        : 'gs://cpg-thousand-genomes-test/gvcf/NA12878-realign.g.vcf.gz',
    'hc_realign'     : 'gs://cpg-thousand-genomes-test/gvcf/NA12878-hc-from-bam.g.vcf.gz',
#     'hc_realign'     : 'gs://cpg-thousand-genomes-test/gvcf/NA12878-hc-realign.g.vcf.gz',
}

# fewgenomes_v2_mt_path = 'gs://cpg-fewgenomes-main/mt/v2-nonref.mt'
# fewgenomes_v2_raw_mt_path = 'gs://cpg-fewgenomes-main-tmp/joint-calling/v2/combiner/v2-raw.mt'
# meta_v2_ht_path = 'gs://cpg-fewgenomes-main-metadata/joint-calling/v2/meta.ht'
# vqsr_final_filter_ht_path = 'gs://cpg-fewgenomes-main-tmp/joint-calling/v2/variant_qc/vqsr/final-filter.ht'
highconf_ht_path = 'gs://cpg-reference/validation/giab/regions/HG001_GRCh38_GIAB_highconf_CG-IllFB-IllGATKHC-Ion-10X-SOLID_CHROM1-X_v.3.3.2_highconf_nosomaticdel_noCENorHET7_hc_regions.ht/'

def filter_highconf(mt):
    highconf_bed_ht = hl.read_table(highconf_ht_path)
    mt = mt.filter_rows(hl.is_defined(highconf_bed_ht[mt.locus]))
    return mt

def print_conc_summary(summary):
    print('Summary:')
    print(summary)
    tp = summary[2][2] + summary[3][3] + summary[4][4]
    total_discordant = sum([sum(s[2:]) for s in summary[2:]]) - tp
    called = sum([sum(s[:]) for s in summary[2:]])
    truth = sum([sum(s[2:]) for s in summary[:]])
    fp = called - tp
    fn = truth - tp
    precision = tp / called
    recall = tp / truth
    print(f'Left = called: {called}')
    print(f'Right = truth: {truth}')
    print(f'Unique to left = FP [precision]: {fp} [{precision:.2%}]')
    print(f'Unique to right = FN [recall]: {fn} [{recall:.2%}]')
    print(f'Concordant = TP [discordant]: {tp} [{total_discordant}]')

Loading BokehJS ...

In [5]:
realign_mt_path = join(work_bucket, 'from_gvcfs', f'hc_realign.mt')
mt = hl.read_matrix_table(realign_mt_path)
mt.count()

(13986427, 1)

In [13]:
realign_mt_path = join(work_bucket, 'from_gvcfs', f'realign.mt')
mt = hl.read_matrix_table(realign_mt_path)
mt.show()

,
locus,alleles
locus<GRCh38>,array<str>
chr1:1,"[""N"",""<NON_REF>""]"
chr1:10013,"[""T"",""<NON_REF>""]"
chr1:10114,"[""T"",""<NON_REF>""]"
chr1:10116,"[""A"",""<NON_REF>""]"
chr1:10120,"[""T"",""<NON_REF>""]"
chr1:10121,"[""A"",""<NON_REF>""]"
chr1:10126,"[""T"",""<NON_REF>""]"
chr1:10127,"[""A"",""<NON_REF>""]"


In [15]:
mt_by_key = dict()
for k, gvcf_path in gvcf_by_key.items():
    mt_path = join(work_bucket, 'from_gvcfs', f'{k}.mt')
    if not utils.file_exists(mt_path):
        mt = hl.import_vcf(gvcf_path, force_bgz=True) 
        mt.write(mt_path, overwrite=True)
    mt = hl.read_matrix_table(mt_path)
    mt_by_key[k] = mt
    print(k)
    print(mt.count())

from_broad_gvcf
(20370777, 1)
from_cram
(20358543, 1)
realign
(19653459, 1)
hc_realign
(13986427, 1)


In [16]:
mt_by_key

{'from_broad_gvcf': <hail.matrixtable.MatrixTable at 0x7ff7f65bdf10>,
 'from_cram': <hail.matrixtable.MatrixTable at 0x7ff7f660e490>,
 'realign': <hail.matrixtable.MatrixTable at 0x7ff7f65ef8b0>,
 'hc_realign': <hail.matrixtable.MatrixTable at 0x7ff7f66f5af0>}

# Compare downloaded with recalled gvcf

In [17]:
mt_by_key = {
    k: hl.split_multi(mt)
    for k, mt in mt_by_key.items()
}

mt_by_key = {
    k: filter_highconf(mt) 
    for k, mt in mt_by_key.items()
}

In [ ]:
{k: mt.show(1) for k, mt in mt_by_key.items()}

In [6]:
{k: mt.count() for k, mt in mt_by_key.items()}

2021-09-23 03:47:39 Hail: INFO: Coerced sorted dataset
2021-09-23 03:48:13 Hail: INFO: Coerced sorted dataset
2021-09-23 03:51:24 Hail: INFO: Coerced sorted dataset
2021-09-23 03:52:03 Hail: INFO: Coerced sorted dataset
2021-09-23 03:55:12 Hail: INFO: Coerced sorted dataset
2021-09-23 03:55:50 Hail: INFO: Coerced sorted dataset
2021-09-23 03:58:18 Hail: INFO: Coerced sorted dataset
2021-09-23 03:58:18 Hail: INFO: Coerced sorted dataset


{'from_broad_gvcf': (14668414, 1),
 'from_cram': (14668278, 1),
 'realign': (14425591, 1),
 'hc_realign': (0, 1)}

In [18]:
truth_mt_path = (
    'gs://cpg-reference/validation/giab/truth/HG001_GRCh38_GIAB_highconf_CG-IllFB-IllGATKHC-Ion-10X-SOLID_CHROM1-X_v.3.3.2_highconf_PGandRTGphasetransfer.mt/'
)
truth_mt = hl.read_matrix_table(truth_mt_path)
truth_mt = truth_mt.key_cols_by(s = truth_mt.s.replace('HG001', TRUTH_SAMPLE))
truth_mt = filter_highconf(truth_mt)
truth_mt = hl.split_multi(truth_mt)

In [29]:
truth_mt.cols().show()

""
s
str
"""NA12878"""


In [5]:
{k: mt.count() for k, mt in mt_by_key.items()}, truth_mt.count()

2021-09-23 03:32:52 Hail: INFO: Coerced sorted dataset
2021-09-23 03:33:06 Hail: INFO: Coerced sorted dataset
2021-09-23 03:34:20 Hail: INFO: Coerced sorted dataset
2021-09-23 03:34:36 Hail: INFO: Coerced sorted dataset
2021-09-23 03:35:50 Hail: INFO: Coerced sorted dataset
2021-09-23 03:36:04 Hail: INFO: Coerced sorted dataset
2021-09-23 03:37:01 Hail: INFO: Coerced sorted dataset
2021-09-23 03:37:01 Hail: INFO: Coerced sorted dataset


({'from_broad_gvcf': (14668414, 1),
  'from_cram': (14668278, 1),
  'realign': (14425591, 1),
  'hc_realign': (0, 1)},
 (3577097, 1))

In [ ]:
hl.filter_intervals(truth_mt, [hl.parse_locus_interval('chr5:1917500-1917600')]).show()

In [30]:
summary, sample_conc_ht, sites_conc_ht = hl.concordance(mt_by_key['from_broad_gvcf'], truth_mt)
print_conc_summary(summary)

2021-09-23 05:35:31 Hail: INFO: concordance: including 1 shared samples (1 total on left, 1 total on right)
2021-09-23 05:35:31 Hail: INFO: Table.join: renamed the following fields on the right to avoid name conflicts:
    'locus' -> 'locus_1'
    'alleles' -> 'alleles_1'


Summary:
[[0, 0, 0, 9761, 7472], [230, 0, 0, 20, 2], [7439147, 0, 0, 0, 0], [2249523, 0, 0, 2128827, 5846], [1419650, 0, 0, 886, 1424283]]
Left = called: 14668162
Right = truth: 3577097
Unique to left = FP [precision]: 11115052 [24.22%]
Unique to right = FN [recall]: 23987 [99.33%]
Concordant = TP [discordant]: 3553110 [6732]


2021-09-23 06:01:07 Hail: INFO: concordance: total concordance 99.81%


In [31]:
summary_from_cram, sample_conc_from_cram_ht, sites_conc_from_cram_ht = hl.concordance(mt_by_key['from_cram'], truth_mt)
print_conc_summary(summary_from_cram)

2021-09-23 06:01:09 Hail: INFO: concordance: including 1 shared samples (1 total on left, 1 total on right)
2021-09-23 06:01:09 Hail: INFO: Table.join: renamed the following fields on the right to avoid name conflicts:
    'locus' -> 'locus_1'
    'alleles' -> 'alleles_1'


Summary:
[[0, 0, 0, 9760, 7473], [5, 0, 0, 0, 1], [7439117, 0, 0, 0, 0], [2249625, 0, 0, 2128847, 5826], [1419667, 0, 0, 887, 1424303]]
Left = called: 14668272
Right = truth: 3577097
Unique to left = FP [precision]: 11115122 [24.22%]
Unique to right = FN [recall]: 23947 [99.33%]
Concordant = TP [discordant]: 3553150 [6713]


2021-09-23 06:06:57 Hail: INFO: concordance: total concordance 99.81%


In [25]:
mt_by_key['hc_realign'].cols().show()
'NA12878-realign'

""
s
str
"""NA12878-hc-from-bam"""


'NA12878-realign'

In [24]:
mt_by_key['realign'] = mt_by_key['realign'].key_cols_by(s = mt_by_key['realign'].s.replace('NA12878-realign', TRUTH_SAMPLE))
summary_realign, sample_conc_realign_ht, sites_conc_realign_ht = hl.concordance(mt_by_key['realign'], truth_mt)
print_conc_summary(summary_realign)

2021-09-23 11:30:28 Hail: INFO: concordance: including 1 shared samples (1 total on left, 1 total on right)
2021-09-23 11:30:28 Hail: INFO: Table.join: renamed the following fields on the right to avoid name conflicts:
    'locus' -> 'locus_1'
    'alleles' -> 'alleles_1'


Summary:
[[0, 0, 0, 47400, 30632], [5, 0, 0, 0, 1], [7340257, 0, 0, 0, 0], [2188999, 0, 0, 2090667, 5656], [1397265, 0, 0, 1427, 1401314]]
Left = called: 14425585
Right = truth: 3577097
Unique to left = FP [precision]: 10933604 [24.21%]
Unique to right = FN [recall]: 85116 [97.62%]
Concordant = TP [discordant]: 3491981 [7083]


2021-09-23 11:38:53 Hail: INFO: concordance: total concordance 99.80%


In [26]:
mt_by_key['hc_realign'] = mt_by_key['hc_realign'].key_cols_by(s = mt_by_key['hc_realign'].s.replace(mt_by_key['hc_realign'].s.collect()[0], TRUTH_SAMPLE))
summary_realign_hc, sample_conc_realign_hc_ht, sites_conc_realign_hc_ht = hl.concordance(mt_by_key['hc_realign'], truth_mt)
print_conc_summary(summary_realign_hc)

2021-09-23 11:38:55 Hail: INFO: concordance: including 1 shared samples (1 total on left, 1 total on right)
2021-09-23 11:38:55 Hail: INFO: Table.join: renamed the following fields on the right to avoid name conflicts:
    'locus' -> 'locus_1'
    'alleles' -> 'alleles_1'


Summary:
[[0, 0, 0, 42762, 25937], [4, 0, 0, 0, 0], [4768219, 0, 0, 0, 0], [2161416, 0, 0, 2095253, 2043], [1392993, 0, 0, 1479, 1409623]]
Left = called: 11831026
Right = truth: 3577097
Unique to left = FP [precision]: 8326150 [29.62%]
Unique to right = FN [recall]: 72221 [97.98%]
Concordant = TP [discordant]: 3504876 [3522]


2021-09-23 11:41:30 Hail: INFO: concordance: total concordance 99.90%


In [20]:
def run_conc(key):
    mt = mt_by_key[key]
    print(f'{key}: running concordance')
    summary, sample_conc_ht, sites_conc_ht = hl.concordance(mt, truth_mt)
    print_conc_summary(summary)
    print(f'{key}: writing summary')
    with hl.hadoop_open(f'{work_bucket}/results/summary_{key}.pickle', 'w') as f:
        f.write(str(summary))
    print(f'{key}: writing sample_conc_ht')
    sample_conc_ht = sample_conc_ht.checkpoint(f'{work_bucket}/results/sample_conc_{key}.ht')
    print(f'{key}: writing sites_conc_ht')
    sites_conc_ht = sites_conc_ht.checkpoint(f'{work_bucket}/results/sites_conc_{key}.ht')

In [21]:
for k in mt_by_key:
    run_conc(k)
    
gnomad_mt = gnomad_mt.key_cols_by(s = gnomad_mt.s.replace('gnomad_sn', 'NA12878'))


from_broad_gvcf: running concordance


2021-09-23 04:54:30 Hail: INFO: concordance: including 0 shared samples (1 total on left, 1 total on right)
2021-09-23 04:54:30 Hail: INFO: Table.join: renamed the following fields on the right to avoid name conflicts:
    'locus' -> 'locus_1'
    'alleles' -> 'alleles_1'


Summary:
[[0, 0, 0, 0, 0], [0, 0, 0, 0, 0], [0, 0, 0, 0, 0], [0, 0, 0, 0, 0], [0, 0, 0, 0, 0]]


2021-09-23 04:56:56 Hail: INFO: concordance: total concordance nan%


ZeroDivisionError: division by zero

In [35]:
print(f'{work_bucket}/results')

gs://cpg-tob-wgs-test-analysis/jupyter/vsavelyev/results


# From fewgenomes joint-calling MT

### Reading the fewgenomes matrix table of 122 samples

In [27]:
mt = utils.get_mt(
    fewgenomes_v2_raw_mt_path, 
    split=True, 
    add_meta=True, 
    meta_ht=hl.read_table(meta_v2_ht_path)
)

### Subsetting to NA12878 - 5m variants

In [28]:
mt = mt.filter_cols(hl.literal([TRUTH_SAMPLE]).contains(mt['s']))
mt = mt.filter_rows((hl.len(mt.alleles) > 1) & (hl.agg.any(mt.GT.is_non_ref())))
mt.count()

(5139649, 1)

### Annotating with VQSR

In [30]:
vqsr_ht = hl.read_table(vqsr_final_filter_ht_path)
mt = mt.annotate_rows(**vqsr_ht[mt.row_key])
mt = mt.annotate_globals(**vqsr_ht.index_globals())

In [33]:
mt.entries().show()

+---------------+------------+------+-------------------+------------+---------+-----------+-----------+----------------------------+
| locus         | alleles    | rsid | n_unsplit_alleles | mixed_site | a_index | was_split |      qual | filters                    |
+---------------+------------+------+-------------------+------------+---------+-----------+-----------+----------------------------+
| locus<GRCh38> | array<str> | str  |             int32 |       bool |   int32 |      bool |   float64 | set<str>                   |
+---------------+------------+------+-------------------+------------+---------+-----------+-----------+----------------------------+
| chr1:10327    | ["T","C"]  | NA   |                 2 |      False |       1 |     False | -1.00e+01 | {"VQSR"}                   |
| chr1:10439    | ["AC","A"] | NA   |                 2 |      False |       1 |     False | -1.00e+01 | {"VQSR"}                   |
| chr1:10492    | ["C","T"]  | NA   |                 2 |      False |       1 |     False | -1.00e+01 | {"VQSR"}                   |
| chr1:13813    | ["T","G"]  | NA   |                 2 |      False |       1 |     False | -1.00e+01 | {"VQSR"}                   |
| chr1:13838    | ["C","T"]  | NA   |                 2 |      False |       1 |     False | -1.00e+01 | {"InbreedingCoeff","VQSR"} |
| chr1:13912    | ["G","A"]  | NA   |                 2 |      False |       1 |     False | -1.00e+01 | {"VQSR"}                   |
| chr1:14248    | ["T","G"]  | NA   |                 2 |      False |       1 |     False | -1.00e+01 | {"VQSR"}                   |
| chr1:14354    | ["C","A"]  | NA   |                 2 |      False |       1 |     False | -1.00e+01 | {"VQSR"}                   |
| chr1:14599    | ["T","A"]  | NA   |                 2 |      False |       1 |     False | -1.00e+01 | {"VQSR"}                   |
| chr1:14604    | ["A","G"]  | NA   |                 2 |      False |       1 |     False | -1.00e+01 | {"VQSR"}                   |
+---------------+------------+------+-------------------+------------+---------+-----------+-----------+----------------------------+

+-------------+--------------+------------+----------+-------------------+----------+---------------+--------------+--------+------+-------+
| allele_type | AS_MQRankSum | AS_pab_max |    AS_QD | AS_ReadPosRankSum |   AS_SOR | n_alt_alleles | variant_type | hapmap | omni | mills |
+-------------+--------------+------------+----------+-------------------+----------+---------------+--------------+--------+------+-------+
| str         |      float64 |    float64 |  float32 |           float64 |  float64 |         int32 | str          |   bool | bool |  bool |
+-------------+--------------+------------+----------+-------------------+----------+---------------+--------------+--------+------+-------+
| "snv"       |     1.07e+00 |   1.00e+00 | 8.68e+00 |         -7.20e-01 | 8.84e-02 |             1 | "snv"        |     NA |   NA |    NA |
| "del"       |    -9.25e-01 |   1.00e+00 | 1.38e+01 |          3.58e-01 | 1.43e+00 |             1 | "indel"      |     NA |   NA |    NA |
| "snv"       |    -1.85e-01 |   1.00e+00 | 9.70e+00 |          2.12e-01 | 6.42e-01 |             1 | "snv"        |     NA |   NA |    NA |
| "snv"       |    -2.04e+00 |   1.00e+00 | 7.59e+00 |          1.43e-01 | 6.70e+00 |             1 | "snv"        |     NA |   NA |    NA |
| "snv"       |    -2.28e+00 |   1.00e+00 | 6.42e+00 |          2.71e-01 | 8.12e+00 |             1 | "snv"        |     NA |   NA |    NA |
| "snv"       |    -1.92e+00 |   1.00e+00 | 5.40e+00 |          7.20e-01 | 5.18e+00 |             1 | "snv"        |     NA |   NA |    NA |
| "snv"       |    -6.74e-01 |   1.00e+00 | 4.04e+00 |          4.06e-01 | 6.42e+00 |             1 | "snv"        |     NA |   NA |    NA |
| "snv"       |    -9.22e-01 |   1.00e+00 | 2.98e+00 |          7.78e-01 | 6.94e+00 |             1 | "snv"        |     NA |   NA |    NA |
| "snv"       |    -2.

### Subsetting to passing variants (5.1m down to 4.3m)

In [48]:
mtf = mt.filter_rows(mt.filters.length() == 0)
mtf.rows().count() / mt.rows().count()

0.8599509421752342

In [35]:
mtf.rows().count()

(4419846, 1)

### Subsetting to chr21 for speed

In [20]:
mt21 = hl.filter_intervals(mt, [hl.parse_locus_interval('chr21')])
mt21f = mt21.filter_rows(mt21.filters.length() == 0)

In [50]:
mt21.rows().count(), mt21f.rows().count(), mt21f.rows().count() / mt21.rows().count()

((90102, 1), (62536, 1), 0.6940578455528179)

### Subsetting to the NA12878 high confidence regions

In [23]:
mt_hc = filter_highconf(mt)
mtf_hc = filter_highconf(mtf)
mt21_hc = filter_highconf(mt21)
mt21f_hc = filter_highconf(mt21f)

2021-07-26 03:14:27 Hail: INFO: Reading table without type imputation
  Loading field 'f0' as type str (user-supplied)
  Loading field 'f1' as type int32 (user-supplied)
  Loading field 'f2' as type int32 (user-supplied)
2021-07-26 03:14:27 Hail: INFO: Reading table without type imputation
  Loading field 'f0' as type str (user-supplied)
  Loading field 'f1' as type int32 (user-supplied)
  Loading field 'f2' as type int32 (user-supplied)
2021-07-26 03:14:28 Hail: INFO: Reading table without type imputation
  Loading field 'f0' as type str (user-supplied)
  Loading field 'f1' as type int32 (user-supplied)
  Loading field 'f2' as type int32 (user-supplied)
2021-07-26 03:14:29 Hail: INFO: Reading table without type imputation
  Loading field 'f0' as type str (user-supplied)
  Loading field 'f1' as type int32 (user-supplied)
  Loading field 'f2' as type int32 (user-supplied)


### Saving to the disk to faster reruns

In [70]:
mt_path = join(work_bucket, 'mt.mt')
mtf_path = join(work_bucket, 'mtf.mt')
mt21_path = join(work_bucket, 'mt21.mt')
mt21f_path = join(work_bucket, 'mt21f.mt')
mt_hc_path = join(work_bucket, 'mt_hc.mt')
mtf_hc_path = join(work_bucket, 'mtf_hc.mt')
mt21_hc_path = join(work_bucket, 'mt21_hc.mt')
mt21f_hc_path = join(work_bucket, 'mt21f_hc.mt')
# mt.write(mt_path, overwrite=True)
# mtf.write(mtf_path, overwrite=True)
# mt21.write(mt21_path, overwrite=True)
# mt21f.write(mt21f_path, overwrite=True)
# mt_hc.write(mt_hc_path, overwrite=True)
# mtf_hc.write(mtf_hc_path, overwrite=True)
# mt21_hc.write(mt21_hc_path, overwrite=True)
# mt21f_hc.write(mt21f_hc_path, overwrite=True)
mt = hl.read_matrix_table(mt_path)
mtf = hl.read_matrix_table(mtf_path)
mt21 = hl.read_matrix_table(mt21_path)
mt21f = hl.read_matrix_table(mt21f_path)
mt_hc = hl.read_matrix_table(mt_hc_path)
mtf_hc = hl.read_matrix_table(mtf_hc_path)
mt21_hc = hl.read_matrix_table(mt21_hc_path)
mt21f_hc = hl.read_matrix_table(mt21f_hc_path)

### Exploring % of filtered and % of HC

There are ~30% of variants outside of the HC regions (5.1m > 3.6m), 
or ~22% (4.3m > 3.3m) for filtered variants only.

In [25]:
mt_hc.rows().count() / mt.rows().count(), \
mtf_hc.rows().count() / mtf.rows().count()

2021-07-26 03:18:16 Hail: INFO: Coerced sorted dataset
2021-07-26 03:18:46 Hail: INFO: Coerced sorted dataset


(0.7008891074079183, 0.7859941273971989)

Percentages are a bit difference for chr21:

In [24]:
mt21_hc.rows().count() / mt21.rows().count(), mt21f_hc.rows().count() / mt21f.rows().count()

2021-07-26 03:15:12 Hail: INFO: Coerced sorted dataset
2021-07-26 03:15:19 Hail: INFO: Coerced sorted dataset


(0.5759694568378061, 0.8136913138032493)

Plotting the filtered variants % per contig

In [31]:
ref = hl.get_reference('GRCh38')
contigs = [c for c in ref.contigs if len(c) <= 5]

In [32]:
total_per_contig = [hl.filter_intervals(mt_hc, [hl.parse_locus_interval(c)]).rows().count() for c in contigs]
filtered_per_contig = [hl.filter_intervals(mtf_hc, [hl.parse_locus_interval(c)]).rows().count() for c in contigs]
gn_total_per_contig = [hl.filter_intervals(gnomad_hc_mt, [hl.parse_locus_interval(c)]).rows().count() for c in contigs]
gn_filtered_per_contig = [hl.filter_intervals(gnomad_hc_mtf, [hl.parse_locus_interval(c)]).rows().count() for c in contigs]

In [59]:
p = figure(x_range=contigs)
p.vbar(x=contigs, top=[
    (float(f) / float(t) * total_per_contig[0]) if t else 0 
    for t, f in zip(total_per_contig, filtered_per_contig)], 
       width=1, color='black', fill_color='white')
p.vbar(x=contigs, top=total_per_contig, width=1, color='blue')
p.vbar(x=contigs, top=filtered_per_contig, width=1, color='red')
reset_output()
output_notebook()
show(p)

Loading BokehJS ...

In [61]:
p = figure(x_range=contigs)
p.vbar(x=contigs, top=[
    (float(f) / float(t) * gn_total_per_contig[0]) if t else 0 
    for t, f in zip(gn_total_per_contig, gn_filtered_per_contig)], 
       width=1, color='black', fill_color='white')
p.vbar(x=contigs, top=gn_total_per_contig, width=1, color='blue')
p.vbar(x=contigs, top=gn_filtered_per_contig, width=1, color='red')
reset_output()
output_notebook()
show(p)

Loading BokehJS ...

### Exploring AS_VQSLOD
AS_VQSLOD goes into very negative values

In [19]:
reset_output()
output_notebook()
show(hl.plot.histogram(mt21.aggregate_entries(hl.expr.aggregators.hist(mt21.AS_VQSLOD, -80, 20, 100))))

Loading BokehJS ...

In [39]:
mt21.aggregate_rows(hl.agg.min(mt21.AS_VQSLOD))

-33583.5947

Now after removing the filtered variants. It's evident that filtering is performed on a certain threshold of the score derived from AS_VQSLOD:

In [20]:
show(hl.plot.histogram(mt21f.aggregate_entries(hl.expr.aggregators.hist(mt21f.AS_VQSLOD, -80, 20, 100))))

### Preprare the truth sample

In [9]:
truth_mt_path = (
    'gs://gnomad-public/resources/grch38/na12878/'
    'HG001_GRCh38_GIAB_highconf_CG-IllFB-IllGATKHC-Ion-10X-SOLID_CHROM1'
    '-X_v.3.3.2_highconf_PGandRTGphasetransfer.mt'
)
truth_mt = hl.read_matrix_table(truth_mt_path)
# truth_mt = truth_mt.key_cols_by(s=hl.str(TRUTH_SAMPLE))
# truth_mt = hl.split_multi_hts(truth_mt, left_aligned=False)
# truth_mt = truth_mt.filter_rows(hl.agg.any(truth_mt.GT.is_non_ref()))
truth_mt.count()

(3659369, 1)

In [10]:
truth_mt21 = hl.filter_intervals(truth_mt, [hl.parse_locus_interval('chr21')])

The truth sample variants sitting almost entirely in the HC regions, however 2.5% are outside:

In [31]:
truth_mt_hc = filter_highconf(truth_mt)
truth_mt_hc.rows().count() / truth_mt.rows().count()

2021-07-26 03:48:36 Hail: INFO: Reading table without type imputation
  Loading field 'f0' as type str (user-supplied)
  Loading field 'f1' as type int32 (user-supplied)
  Loading field 'f2' as type int32 (user-supplied)
2021-07-26 03:48:39 Hail: INFO: Coerced sorted dataset


0.9776527046056301

In [30]:
truth_mt21_hc = filter_highconf(truth_mt21)
truth_mt21_hc.rows().count() / truth_mt21.rows().count()

2021-07-26 03:48:04 Hail: INFO: Reading table without type imputation
  Loading field 'f0' as type str (user-supplied)
  Loading field 'f1' as type int32 (user-supplied)
  Loading field 'f2' as type int32 (user-supplied)
2021-07-26 03:48:07 Hail: INFO: Coerced sorted dataset


0.9752576931049428

Saving on disk

In [4]:
truth_mt_path = join(work_bucket, 'truth_mt_hc.mt')
truth_mt21_path = join(work_bucket, 'truth_mt21_hc.mt')
# truth_mt.write(truth_mt_path)
# truth_mt21.write(truth_mt21_path)
truth_mt = hl.read_matrix_table(truth_mt_path)
truth_mt21 = hl.read_matrix_table(truth_mt21_path)

### Evaluation: concordance with truth

Our matrix table against the truth showed the concodrance of 99.98%, with a bit higher 2% FN rate:

In [6]:
summary21, sample_conc_21ht, sites_conc_21ht = hl.concordance(mt21, truth_mt21)
print_conc_summary(summary21)

2021-07-26 15:04:44 Hail: INFO: concordance: including 1 shared samples (1 total on left, 1 total on right)
2021-07-26 15:04:44 Hail: INFO: Table.join: renamed the following fields on the right to avoid name conflicts:
    'locus' -> 'locus_1'
    'alleles' -> 'alleles_1'


Summary:
[[0, 0, 0, 865, 533], [0, 0, 0, 0, 0], [0, 0, 0, 0, 0], [194, 0, 0, 31564, 5], [33, 0, 0, 6, 20094]]
Left = called: 51896
Right = truth: 53067
Unique to left = FP [precision]: 238 [99.54%]
Unique to right = FN [recall]: 1409 [97.34%]
Concordant = TP [discordant]: 51658 [11]


2021-07-26 15:05:06 Hail: INFO: concordance: total concordance 99.98%
2021-07-26 15:05:06 Hail: WARN: cols(): Resulting column table is sorted by 'col_key'.
    To preserve matrix table column order, first unkey columns with 'key_cols_by()'


In [ ]:
mt_vs_truthmt_summary, sample_concordance_ht, sites_concordance_ht = hl.concordance(mt, truth_mt)
print_conc_summary(mt_vs_truthmt_summary)

### Evaluation: concordnace after filtering to PASS variants

Higher precision, but even lower recall

In [34]:
summary21f, _, _ = hl.concordance(mt21f_hc, truth_mt21)
print_conc_summary(summary21f)

2021-07-26 11:35:55 Hail: INFO: concordance: including 1 shared samples (1 total on left, 1 total on right)
2021-07-26 11:35:55 Hail: INFO: Table.join: renamed the following fields on the right to avoid name conflicts:
    'locus' -> 'locus_1'
    'alleles' -> 'alleles_1'


Summary:
[[0, 0, 0, 1386, 837], [0, 0, 0, 0, 0], [0, 0, 0, 0, 0], [16, 0, 0, 31044, 5], [25, 0, 0, 5, 19790]]
Left = called: 50885
Right = truth: 53067
Unique to left = FP [precision]: 51 [99.90%]
Unique to right = FN [recall]: 2233 [95.79%]
Concordant = TP [discordant]: 50834 [10]


2021-07-26 11:36:00 Hail: INFO: concordance: total concordance 99.98%


In [ ]:
mt_vs_truthmt_summaryf, _, _ = hl.concordance(mtf_hc, truth_mt)
print_conc_summary(mt_vs_truthmt_summaryf)

### Compare with gnomad mt

In [32]:
gnomad_mt_path = 'gs://gcp-public-data--gnomad/release/3.1/mt/genomes/gnomad.genomes.v3.1.hgdp_1kg_subset_dense.mt/'
gnomad_mt = hl.read_matrix_table(gnomad_mt_path)
gnomad_mt.describe()

----------------------------------------
Global fields:
    'global_annotation_descriptions': struct {
        sex_imputation_ploidy_cutoffs: struct {
            Description: str
        }, 
        population_inference_pca_metrics: struct {
            Description: str
        }, 
        hard_filter_cutoffs: struct {
            Description: str
        }, 
        cohort_freq_meta: struct {
            Description: str
        }, 
        gnomad_freq_meta: struct {
            Description: str
        }, 
        cohort_freq_index_dict: struct {
            Description: str
        }, 
        gnomad_freq_index_dict: struct {
            Description: str
        }, 
        gnomad_faf_index_dict: struct {
            Description: str
        }, 
        gnomad_faf_meta: struct {
            Description: str
        }, 
        vep_version: struct {
            Description: str
        }, 
        vep_csq_header: struct {
            Description: str
        }, 
        dbsnp_versio

In [ ]:
gnomad_sn = 'v3.1::NA12878'
gnomad_mt = gnomad_mt.key_cols_by(s = gnomad_mt.s.replace(gnomad_sn, TRUTH_SAMPLE))
gnomad_mt = gnomad_mt.filter_cols(hl.literal([TRUTH_SAMPLE]).contains(gnomad_mt['s']))
gnomad_mt = gnomad_mt.filter_rows((hl.len(gnomad_mt.alleles) > 1) & (hl.agg.any(gnomad_mt.GT.is_non_ref())))
gnomad_mt.count()

Subset to PASSing

In [5]:
gnomad_mtf = gnomad_mt.filter_rows(gnomad_mt.filters.length() == 0)
gnomad_mt21 = hl.filter_intervals(gnomad_mt, [hl.parse_locus_interval('chr21')])
gnomad_mt21f = gnomad_mt21.filter_rows(gnomad_mt21.filters.length() == 0)

Are there multiallelics? - no

In [ ]:
#gnomad_mt.filter_rows(hl.len(gnomad_mt.alleles) > 2).rows().count()

### Subset to highconf regions

In [ ]:
gnomad_hc_mt = filter_highconf(gnomad_mt)
gnomad_hc_mtf =  gnomad_hc_mt.filter_rows(gnomad_hc_mt.filters.length() == 0)
gnomad_hc_mt21 = hl.filter_intervals(gnomad_hc_mt, [hl.parse_locus_interval('chr21')])
gnomad_hc_mt21f = gnomad_mt21_hc.filter_rows(gnomad_hc_mt21.filters.length() == 0)

Saving on disk

In [20]:
gnomad_mt_path = join(work_bucket, 'gnomad.mt')
gnomad_mtf_path = join(work_bucket, 'gnomad.filt.mt')
gnomad_mt21_path = join(work_bucket, 'gnomad.chr21.mt')
gnomad_mt21f_path = join(work_bucket, 'gnomad.chr21.filt.mt')
gnomad_hc_mt_path = join(work_bucket, 'gnomad_hc.mt')
gnomad_hc_mtf_path = join(work_bucket, 'gnomad_hc.filt.mt')
gnomad_hc_mt21_path = join(work_bucket, 'gnomad_hc.chr21.mt')
gnomad_hc_mt21f_path = join(work_bucket, 'gnomad_hc.chr21.filt.mt')
# gnomad_mt.write(gnomad_mt_path, overwrite=True)
# gnomad_mtf.write(gnomad_mtf_path, overwrite=True)
# gnomad_mt21.write(gnomad_mt21_path, overwrite=True)
# gnomad_mt21f.write(gnomad_mt21f_path, overwrite=True)
# gnomad_mt_hc.write(gnomad_hc_mt_path, overwrite=True)
# gnomad_mtf_hc.write(gnomad_hc_mtf_path, overwrite=True)
# gnomad_mt21_hc.write(gnomad_hc_mt21_path, overwrite=True)
# gnomad_mt21f_hc.write(gnomad_hc_mt21f_path, overwrite=True)
gnomad_mt = hl.read_matrix_table(gnomad_mt_path)
gnomad_mtf = hl.read_matrix_table(gnomad_mtf_path)
gnomad_mt21 = hl.read_matrix_table(gnomad_mt21_path)
gnomad_mt21f = hl.read_matrix_table(gnomad_mt21f_path)
gnomad_hc_mt = hl.read_matrix_table(gnomad_hc_mt_path)
gnomad_hc_mtf = hl.read_matrix_table(gnomad_hc_mtf_path)
gnomad_hc_mt21 = hl.read_matrix_table(gnomad_hc_mt21_path)
gnomad_hc_mt21f = hl.read_matrix_table(gnomad_hc_mt21f_path)

92% of gnomAD variants are PASSing, with 99% in the HC regions.

In [25]:
gnomad_mtf.rows().count(), gnomad_mt.rows().count(), \
gnomad_mtf.rows().count() / gnomad_mt.rows().count(), \
gnomad_hc_mtf.rows().count(), gnomad_hc_mt.rows().count(), \
gnomad_hc_mtf.rows().count() / gnomad_hc_mt.rows().count()

(4640374, 4995085, 0.9289879951992809, 3559009, 3597730, 0.9892373802369827)

75% though in chr21:

In [28]:
gnomad_mt21f.rows().count(), gnomad_mt21.rows().count(), \
gnomad_mt21f.rows().count() / gnomad_mt21.rows().count(), \
gnomad_hc_mt21f.rows().count(), gnomad_hc_mt21.rows().count(), \
gnomad_hc_mt21f.rows().count() / gnomad_hc_mt21.rows().count()

(67677, 89533, 0.7558888901298962, 51537, 51852, 0.9939250173570933)

### Exploring AS_VQSLOD in Gnomad MT

In [36]:
reset_output()
output_notebook()
show(hl.plot.histogram(gnomad_mt21.aggregate_entries(hl.expr.aggregators.hist(gnomad_mt21.vqsr.AS_VQSLOD, -80, 30, 100))))

Loading BokehJS ...

In [37]:
show(hl.plot.histogram(gnomad_mt21f.aggregate_entries(hl.expr.aggregators.hist(gnomad_mt21f.vqsr.AS_VQSLOD, -80, 30, 100))))

### Evaluation: gnomad MT vs truth MT

In [8]:
gn21_summary, gn_sample_conc_21ht, gn_sites_conc_21ht = hl.concordance(gnomad_hc_mt21, truth_mt21)
print_conc_summary(gn21_summary)

2021-07-26 15:06:16 Hail: INFO: concordance: including 1 shared samples (1 total on left, 1 total on right)
2021-07-26 15:06:16 Hail: INFO: Table.join: renamed the following fields on the right to avoid name conflicts:
    'locus' -> 'locus_1'
    'alleles' -> 'alleles_1'


Summary:
[[0, 0, 0, 870, 533], [0, 0, 0, 0, 0], [0, 0, 0, 0, 0], [155, 0, 0, 31558, 5], [33, 0, 0, 7, 20094]]
Left = called: 51852
Right = truth: 53067
Unique to left = FP [precision]: 200 [99.61%]
Unique to right = FN [recall]: 1415 [97.33%]
Concordant = TP [discordant]: 51652 [12]


2021-07-26 15:06:26 Hail: INFO: concordance: total concordance 99.98%


In [ ]:
gn_summary, gn_sample_concordance_ht, gn_sites_concordance_ht = hl.concordance(gnomad_hc_mt, truth_mt)
print_conc_summary(gn_summary)

In [41]:
gnomad_mt21f.cols().show()

+-----------------+---------------------------+--------------------------+---------------------+---------------------------+
| s               | bam_metrics.pct_bases_20x | bam_metrics.pct_chimeras | bam_metrics.freemix | bam_metrics.mean_coverage |
+-----------------+---------------------------+--------------------------+---------------------+---------------------------+
| str             |                   float64 |                  float64 |             float64 |                   float64 |
+-----------------+---------------------------+--------------------------+---------------------+---------------------------+
| "v3.1::NA12878" |                  9.47e+01 |                 6.83e-01 |            3.00e-06 |                  3.14e+01 |
+-----------------+---------------------------+--------------------------+---------------------+---------------------------+

+-----------------------------+------------------------------+--------------------------------+---------------------------+-------------+
| bam_metrics.median_coverage | bam_metrics.mean_insert_size | bam_metrics.median_insert_size | bam_metrics.pct_bases_10x | subsets.tgp |
+-----------------------------+------------------------------+--------------------------------+---------------------------+-------------+
|                     float64 |                      float64 |                        float64 |                   float64 |        bool |
+-----------------------------+------------------------------+--------------------------------+---------------------------+-------------+
|                    3.20e+01 |                     4.42e+02 |                       4.33e+02 |                  9.71e+01 |        True |
+-----------------------------+------------------------------+--------------------------------+---------------------------+-------------+

+--------------+------------------------------+-----------------------------+-----------------------------+----------------------------+
| subsets.hgdp | sex_imputation.chr20_mean_dp | sex_imputation.chrX_mean_dp | sex_imputation.chrY_mean_dp | sex_imputation.chrX_ploidy |
+--------------+------------------------------+-----------------------------+-----------------------------+----------------------------+
|         bool |                      float32 |                     float32 |                     float32 |                    float32 |
+--------------+------------------------------+-----------------------------+-----------------------------+----------------------------+
|        False |                     3.25e+01 |                    2.98e+01 |                    8.12e-01 |                   1.83e+00 |
+--------------+------------------------------+-----------------------------+-----------------------------+----------------------------+

+----------------------------+----------------------------+----------------------------+------------------------------+
| sex_imputation.chrY_ploidy | sex_imputation.X_karyotype | sex_imputation.Y_karyotype | sex_imputation.sex_karyotype |
+----------------------------+----------------------------+----------------------------+------------------------------+
|                    float32 | str                        | str                        | str                          |
+----------------------------+----------------------------+----------------------------+------------------------------+
|                   5.01e-02 | "XX"                       | ""                         | "XX"                         |
+----------------------------+----------------------------+----------------------------+------------------------------+

+----------------------------------------+------------------------------------------+------------------------------------------+
| sex_imputation.impute_sex_stats.f_stat | sex_imputation.impute_sex_stats.n_called | sex_imputation.impute_sex_stats.expec... |
+----------------------------------------+------------------------------------------+-------------

Filtered gnomad MT vs truth

In [39]:
gnf21_summary, _, g_ = hl.concordance(gnomad_hc_mt21f, truth_mt21)
print_conc_summary(gnf21_summary)

2021-07-26 11:40:37 Hail: INFO: concordance: including 1 shared samples (1 total on left, 1 total on right)
2021-07-26 11:40:38 Hail: INFO: Table.join: renamed the following fields on the right to avoid name conflicts:
    'locus' -> 'locus_1'
    'alleles' -> 'alleles_1'


Summary:
[[0, 0, 0, 994, 586], [0, 0, 0, 0, 0], [0, 0, 0, 0, 0], [18, 0, 0, 31435, 4], [32, 0, 0, 6, 20042]]
Left = called: 51537
Right = truth: 53067
Unique to left = FP [precision]: 60 [99.88%]
Unique to right = FN [recall]: 1590 [97.00%]
Concordant = TP [discordant]: 51477 [10]


2021-07-26 11:40:51 Hail: INFO: concordance: total concordance 99.98%


In [ ]:
gnffull_truth_summary, _, _ = hl.concordance(gnomad_hc_mtf, truth_mt)
print_conc_summary(gnffull_truth_summary)

### Evaluation: filtered gnomad MT vs filtered MT

In [40]:
gnf_summary, _, _ = hl.concordance(gnomad_hc_mt21f, mt21f_hc)
print_conc_summary(gnf_summary)

2021-07-26 11:40:52 Hail: INFO: concordance: including 1 shared samples (1 total on left, 1 total on right)
2021-07-26 11:40:52 Hail: INFO: Table.join: renamed the following fields on the right to avoid name conflicts:
    'inbreeding_coeff_cutoff' -> 'inbreeding_coeff_cutoff_1'
    'filtering_model' -> 'filtering_model_1'
    'locus' -> 'locus_1'
    'alleles' -> 'alleles_1'


Summary:
[[0, 0, 0, 93, 30], [0, 0, 0, 0, 0], [0, 0, 0, 0, 0], [486, 0, 0, 30971, 0], [289, 0, 0, 1, 19790]]
Left = called: 51537
Right = truth: 50885
Unique to left = FP [precision]: 776 [98.49%]
Unique to right = FN [recall]: 124 [99.76%]
Concordant = TP [discordant]: 50761 [1]


2021-07-26 11:41:04 Hail: INFO: concordance: total concordance 100.00%


In [ ]:
gnffull_summary, _, _ = hl.concordance(gnomad_hc_mtf, mtf_hc)
print_conc_summary(gnffull_summary)

# Binned concordance

In [71]:
_, gn_sample_conc_ht, gn_sites_conc_ht = hl.concordance(gnomad_hc_mt, truth_mt)
_, samples_conc_ht, sites_conc_ht = hl.concordance(mt_hc, truth_mt)

2021-07-27 07:35:06 Hail: INFO: concordance: including 0 shared samples (1 total on left, 1 total on right)
2021-07-27 07:35:06 Hail: INFO: Table.join: renamed the following fields on the right to avoid name conflicts:
    'locus' -> 'locus_1'
    'alleles' -> 'alleles_1'
2021-07-27 07:37:28 Hail: INFO: concordance: total concordance nan%
2021-07-27 07:37:28 Hail: WARN: cols(): Resulting column table is sorted by 'col_key'.
    To preserve matrix table column order, first unkey columns with 'key_cols_by()'
2021-07-27 07:37:29 Hail: INFO: concordance: including 1 shared samples (1 total on left, 1 total on right)
2021-07-27 07:37:29 Hail: INFO: Table.join: renamed the following fields on the right to avoid name conflicts:
    'locus' -> 'locus_1'
    'alleles' -> 'alleles_1'
2021-07-27 07:37:44 Hail: INFO: concordance: total concordance 99.96%


2021-07-27 07:37:44 Hail: INFO: concordance: including 1 shared samples (1 total on left, 1 total on right)
2021-07-27 07:37:45 Hail: INFO: Table.join: renamed the following fields on the right to avoid name conflicts:
    'locus' -> 'locus_1'
    'alleles' -> 'alleles_1'
2021-07-27 07:38:04 Hail: INFO: concordance: total concordance 99.96%


In [73]:
sites_conc_ht_path = join(work_bucket, 'sites_concordance.ht')
samples_conc_ht_path = join(work_bucket, 'sample_concordance.ht')
gn_sites_conc_ht_path = join(work_bucket, 'gn_sites_concordance.ht')
gn_samples_conc_ht_path = join(work_bucket, 'gn_sample_concordance.ht')
sites_conc_ht.write(sites_conc_ht_path, overwrite=True)
sample_conc_ht.write(samples_conc_ht_path, overwrite=True)
gn_sites_conc_ht.write(gn_sites_conc_ht_path, overwrite=True)
gn_sample_conc_ht.write(gn_samples_conc_ht_path, overwrite=True)

2021-07-27 07:38:37 Hail: INFO: wrote table with 3687842 rows in 9718 partitions to gs://cpg-tob-wgs-test-tmp/concordance/tmp/sites_concordance.ht
    Total size: 61.72 MiB
    * Rows: 61.72 MiB
    * Globals: 235.00 B
    * Smallest partition: 0 rows (21.00 B)
    * Largest partition:  1979 rows (31.00 KiB)


NameError: name 'sample_conc_ht' is not defined

In [ ]:
sites_conc_ht = hl.read_table(sites_conc_ht_path)
sample_conc_ht = hl.read_table(sample_conc_ht_path)
gn_sites_conc_ht = hl.read_table(gn_sites_conc_ht_path)
gn_sample_conc_ht = hl.read_table(gn_sample_conc_ht_path)

### Rank variants (sort by the score)

In [80]:
gnomad_mt21.describe()

----------------------------------------
Global fields:
    'global_annotation_descriptions': struct {
        sex_imputation_ploidy_cutoffs: struct {
            Description: str
        }, 
        population_inference_pca_metrics: struct {
            Description: str
        }, 
        hard_filter_cutoffs: struct {
            Description: str
        }, 
        cohort_freq_meta: struct {
            Description: str
        }, 
        gnomad_freq_meta: struct {
            Description: str
        }, 
        cohort_freq_index_dict: struct {
            Description: str
        }, 
        gnomad_freq_index_dict: struct {
            Description: str
        }, 
        gnomad_faf_index_dict: struct {
            Description: str
        }, 
        gnomad_faf_meta: struct {
            Description: str
        }, 
        vep_version: struct {
            Description: str
        }, 
        vep_csq_header: struct {
            Description: str
        }, 
        dbsnp_versio

In [63]:
def rank_variants(
    mt,
    model='CPG-AS-VQSR', 
    truth_sample=TRUTH_SAMPLE,
    overwrite=True
):
    score_rank_ht_path = join(work_bucket, 'vqsr', model, truth_sample, 'score_rank.ht')
    if not overwrite and utils.file_exists(score_rank_ht_path):
        score_rank_ht = hl.read_table(score_rank_ht_path)
        return score_rank_ht
    print(f"Creating rank file for VQSR")
    ht = mt.rows()
    print('Filtering to high_quality samples and n_nonref==1...')
    if 'was_split' not in ht._fields:
        ht = ht.annotate(was_split = True)
    if 'singleton' not in ht._fields:
        ht = ht.annotate(singleton = True)
    if 'ac' not in ht._fields:
        ht = ht.annotate(ac = ht.gnomad_popmax.AC)
    ht = ht.select(
        was_split=ht.was_split,
        singleton=ht.singleton,
        ac=ht.ac,
        score=ht.vqsr.AS_VQSLOD, 
        negative_train_site=ht.vqsr.NEGATIVE_TRAIN_SITE,
        positive_train_site=ht.vqsr.POSITIVE_TRAIN_SITE,
        culprit=ht.vqsr.AS_culprit
    )

    ht = add_rank(
        ht,
        score_expr=-1 * ht.score,
        subrank_expr={
            'biallelic_rank': ~ht.was_split,
            'biallelic_singleton_rank': ~ht.was_split & ht.singleton,
            'adj_rank': ht.ac > 0,
            'adj_biallelic_rank': ~ht.was_split & (ht.ac > 0),
            'adj_singleton_rank': ht.singleton & (ht.ac > 0),
            'adj_biallelic_singleton_rank': ~ht.was_split & ht.singleton & (ht.ac > 0)
        }
    )
    ht.write(score_rank_ht_path, overwrite=True)
    score_rank_ht = hl.read_table(score_rank_ht_path)
    return score_rank_ht

In [90]:
cpg_score_rank_21ht = rank_variants(
    mt21,
    model='CPG-AS-VQSR', 
    truth_sample=TRUTH_SAMPLE,
    overwrite=True
)
gnomad_score_rank_21ht = rank_variants(
    gnomad_mt21,
    model='gnomAD-AS-VQSR', 
    truth_sample=TRUTH_SAMPLE,
    overwrite=True
)

Creating rank file for VQSR
Filtering to high_quality samples and n_nonref==1...


,,
locus,alleles,ac
locus<GRCh38>,array<str>,int32
chr21:10270435,"[""G"",""A""]",3541
chr21:10270437,"[""C"",""G""]",3400
chr21:10270443,"[""T"",""A""]",3992
chr21:10270453,"[""C"",""A""]",3777
chr21:10270455,"[""A"",""C""]",3680
chr21:10270463,"[""T"",""C""]",15347
chr21:10270466,"[""T"",""G""]",3594
chr21:10270468,"[""C"",""T""]",3338


2021-07-26 15:58:53 Hail: INFO: Ordering unsorted dataset with network shuffle
2021-07-26 15:58:56 Hail: INFO: Ordering unsorted dataset with network shuffle
2021-07-26 15:59:23 Hail: INFO: wrote table with 51852 rows in 1461 partitions to gs://cpg-tob-wgs-test-tmp/concordance/tmp/vqsr/gnomAD-AS-VQSR/NA12878/score_rank.ht
    Total size: 1.99 MiB
    * Rows: 1.98 MiB
    * Globals: 7.12 KiB
    * Smallest partition: 0 rows (21.00 B)
    * Largest partition:  179 rows (6.13 KiB)


In [64]:
cpg_score_rank_ht = rank_variants(
    mt,
    model='CPG-AS-VQSR', 
    truth_sample=TRUTH_SAMPLE,
    overwrite=True
)
gnomad_score_rank_ht = rank_variants(
    gnomad_mt,
    model='gnomAD-AS-VQSR', 
    truth_sample=TRUTH_SAMPLE,
    overwrite=True
)

Creating rank file for VQSR
Filtering to high_quality samples and n_nonref==1...


2021-07-27 06:46:39 Hail: INFO: Ordering unsorted dataset with network shuffle
2021-07-27 06:47:01 Hail: INFO: Ordering unsorted dataset with network shuffle
2021-07-27 06:47:50 Hail: INFO: wrote table with 3602324 rows in 9618 partitions to gs://cpg-tob-wgs-test-tmp/concordance/tmp/vqsr/CPG-AS-VQSR/NA12878/score_rank.ht
    Total size: 133.35 MiB
    * Rows: 133.35 MiB
    * Globals: 235.00 B
    * Smallest partition: 0 rows (21.00 B)
    * Largest partition:  1963 rows (74.99 KiB)


Creating rank file for VQSR
Filtering to high_quality samples and n_nonref==1...


2021-07-27 06:48:33 Hail: INFO: Ordering unsorted dataset with network shuffle
2021-07-27 06:50:31 Hail: INFO: Ordering unsorted dataset with network shuffle
2021-07-27 06:55:15 Hail: INFO: wrote table with 4995085 rows in 115375 partitions to gs://cpg-tob-wgs-test-tmp/concordance/tmp/vqsr/gnomAD-AS-VQSR/NA12878/score_rank.ht
    Total size: 200.44 MiB
    * Rows: 200.44 MiB
    * Globals: 7.12 KiB
    * Smallest partition: 0 rows (21.00 B)
    * Largest partition:  1620 rows (56.42 KiB)


### Create binned concordance

In [67]:
nbins = 100

In [68]:
def binned_concordance(
    score_rank_ht,
    sites_concordance_ht,
    model='CPG-AS-VQSR', 
    truth_sample=TRUTH_SAMPLE,
    overwrite=False,
):
    binned_concordance_ht_path = join(work_bucket, 'vqsr', model, truth_sample, 'binned_concordance.ht')
    if not overwrite and utils.file_exists(binned_concordance_ht_path):
        ht = hl.read_table(binned_concordance_ht_path)
        return ht

    ht = sites_concordance_ht  # union of mt and truth_mt sites
    metric_ht = score_rank_ht

    # Total number of SNPs and indels in the target matrix table
    metric_snvs, metrics_indels = metric_ht.aggregate([
        hl.agg.count_where(hl.is_snp(metric_ht.alleles[0], metric_ht.alleles[1])),
        hl.agg.count_where(~hl.is_snp(metric_ht.alleles[0], metric_ht.alleles[1]))
    ])

    # Total number of SNPs and indels in the union
    snvs, indels = ht.aggregate([
        hl.agg.count_where(hl.is_snp(ht.alleles[0], ht.alleles[1])),
        hl.agg.count_where(~hl.is_snp(ht.alleles[0], ht.alleles[1]))
    ])

    ht = ht.annotate_globals(
        global_counts=hl.struct(snvs=metric_snvs, indels=metrics_indels),
        counts=hl.struct(snvs=snvs, indels=indels)
    )
    
    # Annotating the union table with the target table annotations 
    # (so the variants unique to truth won't have those score annotations)
    ht = ht.annotate(
        snv=hl.is_snp(ht.alleles[0], ht.alleles[1]),
        score=metric_ht[ht.key].score,
        global_rank=metric_ht[ht.key].rank,
        # TP => allele is found in both data sets
        n_tp=ht.concordance[3][3] + ht.concordance[3][4] + ht.concordance[4][3] + ht.concordance[4][4],
        # FP => allele is found only in test data set
        n_fp=hl.sum(ht.concordance[3][:2]) + hl.sum(ht.concordance[4][:2]),
        # FN => allele is found only in truth data set
        n_fn=hl.sum(ht.concordance[:2].map(lambda x: x[3] + x[4]))
    )
    
    # Add ranks
    ht = add_rank(ht, -1.0*ht.score)
    ht = ht.annotate(rank=[
        hl.tuple([
            'global_rank', 
            (ht.global_rank + 1) / hl.cond(
                 ht.snv,
                 ht.globals.global_counts.snvs,
                 ht.globals.global_counts.indels
             )
        ]),
        hl.tuple([
            'truth_sample_rank', 
            (ht.rank + 1) / hl.cond(
                 ht.snv,
                 ht.globals.counts.snvs,
                 ht.globals.counts.indels
             )
        ])
    ])
    ht = ht.explode(ht.rank)
    
    ht = ht.annotate(
        rank_name=ht.rank[0],
        bin=hl.int(ht.rank[1] * nbins)
    )

    ht = ht.group_by(
        'rank_name',
        'snv',
        'bin'
    ).aggregate(
        # Look at site-level metrics -> tp > fp > fn -- only important for multi-sample comparisons
        tp=hl.agg.count_where(ht.n_tp > 0),
        fp=hl.agg.count_where((ht.n_tp == 0) & (ht.n_fp > 0)),
        fn=hl.agg.count_where((ht.n_tp == 0) & (ht.n_fp == 0) & (ht.n_fn > 0)),
        min_score=hl.agg.min(ht.score),
        max_score=hl.agg.max(ht.score),
        n_alleles=hl.agg.count()
    ).repartition(5)
    
    ht.show()

    ht.write(binned_concordance_ht_path, overwrite=True)    
    return ht

In [69]:
binned_concordance(
    cpg_score_rank_ht, 
    sites_conc_ht, 
    model='CPG-AS-VQSR', 
    truth_sample=TRUTH_SAMPLE,
    overwrite=True
)
binned_concordance(
    gnomad_score_rank_ht, 
    gn_sites_conc_ht, 
    model='gnomAD-AS-VQSR', 
    truth_sample=TRUTH_SAMPLE,
    overwrite=True
)

NameError: name 'sites_conc_ht' is not defined

In [78]:
ht.show(50)

2021-07-20 10:51:39 Hail: INFO: Ordering unsorted dataset with network shuffle


,,,,,,,,
rank_name,snv,bin,tp,fp,fn,min_score,max_score,n_alleles
str,bool,int32,int64,int64,int64,float64,float64,int64
"""global_rank""",False,0,166542,40866,0,3.65e+00,1.80e+01,207408
"""global_rank""",False,1,107259,100150,0,-6.91e+02,3.65e+00,207409
"""global_rank""",False,2,99761,107648,0,NA,NA,207409
"""global_rank""",False,3,96278,111131,0,NA,NA,207409
"""global_rank""",False,4,97126,110283,0,NA,NA,207409
"""global_rank""",False,5,0,1,0,NA,NA,1
"""global_rank""",False,NA,0,0,4096,NA,NA,4096
"""global_rank""",True,0,745416,75104,0,5.88e+00,6.85e+00,820520


In [3]:
def make_binned_concordance_pd(
    work_bucket: str,
    model_names: List[str],
    truth_samples: List[str],
) -> pd.DataFrame:
    """
    Creates a pandas DF containing the binned concordance results for all given truth samples / models.
    :param list of str truth_samples: List of truth samples to include
    :param list of str or dict of str -> str models: 
        Models to include. Either a list of the model ids, 
        or a dict with model id -> model name for display
    :return: Pandas dataframe with binned concordance results
    :rtype: DataFrame
    """
    
    def get_binned_concordance_ht(model_name, truth_sample):
        ht = hl.read_table(join(work_bucket, 'vqsr', model_name, truth_sample, 'binned_concordance.ht'))
        try:
            ht = ht.drop('global_annotation_descriptions')
        except:
            pass
        ht = ht.annotate_globals(
            filtering_model = ht.filtering_model.annotate(
                snv_cutoff = ht.filtering_model.snv_cutoff.annotate(
                    bin = hl.float(ht.filtering_model.snv_cutoff.bin)
                ),
                indel_cutoff = ht.filtering_model.indel_cutoff.annotate(
                    bin = hl.float(ht.filtering_model.indel_cutoff.bin)
                ),
            )
        )    
        return ht
    
    # Combine binned concordance results for multiple truth samples and/or models into a single Table.
    hts = []
    for truth_sample in truth_samples:
        for model_name in model_names:
            ht = get_binned_concordance_ht(model_name, truth_sample)
            ht = ht.annotate(truth_sample=truth_sample, model=model_name)
            hts.append(ht)
    ht = hts[0].union(*hts[1:])

    def f_score(n, df):
        return (1 + n**2) * df['precision'] * df['recall'] / (n**2 * df['precision'] + df['recall'])

    def compute_cumul_metrics(df: pd.DataFrame) -> pd.DataFrame:
        """
        Computes cumulative metrics on a pandas DF.
        """
        df = df.sort_values(by=['bin'])
        df['cum_tp'] = df['tp'].cumsum()
        df['cum_fp'] = df['fp'].cumsum()
        total_true = df['tp'].sum() + df['fn'].sum()
        total_false = df['fp'].sum()
        df['cum_fn'] = total_true - df['cum_tp']
        df['cum_tn'] = total_false - df['cum_fp']
        df['precision'] = df['cum_tp'] / (df['cum_tp'] + df['cum_fp'])
        df['recall'] = df['cum_tp'] / (df['cum_tp'] + df['cum_fn'])
        df['cum_alleles'] = df['n_alleles'].cumsum()
        df['f1'] = f_score(2, df)
        df['f2'] = f_score(2, df)
        df['f3'] = f_score(3, df)
        df['f4'] = f_score(4, df)
        df['f10'] = f_score(10, df)
        return df[['bin', 'min_score', 'max_score', 'n_alleles', 'tp', 'fp', 'fn', 'cum_alleles', 
                   'cum_tp', 'cum_fp', 'cum_fn', 'cum_tn', 'precision', 'recall', 
                   'f1', 'f2', 'f3', 'f4', 'f10']]

    df = ht.to_pandas()
    df = df.groupby(['rank_name', 'truth_sample', 'model', 'snv']).apply(compute_cumul_metrics)
    return df.fillna(-1).groupby(['rank_name', 'truth_sample', 'model', 'snv'])

df = make_binned_concordance_pd(
    work_bucket,
    model_names=['CPG-AS-VQSR', 'gnomAD-AS-VQSR'],
    truth_samples=[TRUTH_SAMPLE],
)

In [29]:
ht.show(50)

,,,,,,,,,,
rank_name,snv,bin,tp,fp,fn,min_score,max_score,n_alleles,truth_sample,model
str,bool,int32,int64,int64,int64,float64,float64,int64,str,str
"""global_rank""",False,0,68,0,0,1.80e+01,1.80e+01,68,"""NA12878""","""CPG-AS-VQSR"""
"""global_rank""",False,0,68,0,0,1.80e+01,1.80e+01,68,"""NA12878""","""gnomAD-AS-VQSR"""
"""global_rank""",False,1,80,0,0,1.79e+01,1.80e+01,80,"""NA12878""","""CPG-AS-VQSR"""
"""global_rank""",False,1,80,0,0,1.79e+01,1.80e+01,80,"""NA12878""","""gnomAD-AS-VQSR"""
"""global_rank""",False,2,97,0,0,1.79e+01,1.79e+01,97,"""NA12878""","""CPG-AS-VQSR"""
"""global_rank""",False,2,97,0,0,1.79e+01,1.79e+01,97,"""NA12878""","""gnomAD-AS-VQSR"""
"""global_rank""",False,3,74,0,0,1.78e+01,1.79e+01,74,"""NA12878""","""CPG-AS-VQSR"""
"""global_rank""",False,3,74,0,0,1.78e+01,1.79e+01,74,"""NA12878""","""gnomAD-AS-VQSR"""


In [ ]:
df.head()

In [9]:
qc_plots_settings = {
    'mean_point_size': 4.0,
    'min_point_size': 1.0,
    'max_point_size': 16.0,
    'label_text_font_size': "14pt",
    'title.text_font_size': "16pt",
    'subtitle.text_font_size': "14pt",
    'axis.axis_label_text_font_size': "16pt",
    'axis.axis_label_text_font_style': "normal",
    'axis.major_label_text_font_size': "14pt"
}

def set_plots_defaults(p: Plot) -> None:
    p.legend.label_text_font_size = qc_plots_settings['label_text_font_size']
    p.title.text_font_size = qc_plots_settings['title.text_font_size']
    p.axis.axis_label_text_font_size = qc_plots_settings['axis.axis_label_text_font_size']
    p.axis.axis_label_text_font_style = qc_plots_settings['axis.axis_label_text_font_style']
    p.axis.major_label_text_font_size = qc_plots_settings['axis.major_label_text_font_size']

def get_point_size_col(data: pd.Series, size_prop: str) -> pd.Series:
    """
    Given a data Series, returns the corresponding point size either:
    - Constant to qc_plots_settings['mean_point_size'] if `size_prop` is None
    - Radius proportional to data, if `size_prop` is 'radius'
    - Area proportional to data, if `size_prop` is 'area'
    Mean, min and max point  sizes are extracted from qc_plots_settings
    :param Series data: Input data series
    :param str size_prop: One of None, 'radius' or 'area'
    :return: Series with corresponding point size for each data point
    :rtype: Series
    """
    if size_prop is None:
        return pd.Series(len(data) * [qc_plots_settings['mean_point_size']])
    else:
        mean_data = np.mean(data)
        if size_prop == 'radius':
            return data.apply(lambda x: max(qc_plots_settings['min_point_size'], 
                                            min(qc_plots_settings['max_point_size'], 
                                                qc_plots_settings['mean_point_size'] * (x / mean_data))))
        elif size_prop == 'area':
            return data.apply(
                lambda x: max(
                    qc_plots_settings['min_point_size'], 
                    min(
                        qc_plots_settings['max_point_size'], 
                        qc_plots_settings['mean_point_size'] * np.pi * (np.sqrt(x / mean_data) / np.pi)
                    )
                )
            )
        else:
            raise ValueError(f"{size_prop} is not a supported value for argument `size_prop`")
            
def plot_concordance_pr(
    pr_df: pd.DataFrame,
    snv: bool,
    colors: Dict[str, str] = None,
    size_prop: str = None,
    bins_to_label: List[int] = None
) -> Column:
    """
    Generates plots showing Precision/Recall curves for truth samples:
    Two tabs:
    - One displaying the PR curve with ranking computed on the entire data
    - One displaying the PR curve with ranking computed on the truth sample only

    Within each tab, a row of n_truth_samples.
    The input to this function should come out of the `get_binned_concordance_pd` function, 
    which creates  a DataFrame containing the necessary metris for PR plotting and is grouped 
    by 'rank_name', 'truth_sample', 'model' and 'snv'.
    :param DataFrame pr_df: 
           Input Dataframe
    :param bool snv: 
           Whether to plot SNVs or Indels
    :param dict of str -> str colors: 
           Optional colors to use (model name -> desired color)
    :param str size_prop: 
           Either 'radius' or 'area' can be specified. If either is specified, 
           the points will be sized proportionally to the amount of data in that point.
    :param list of int bins_to_label: 
           Bins to label
    :return Bokeh grid of plots
    :rtype Tabs
    """

    if colors is None:
        # Get a palette automatically
        from bokeh.palettes import d3
        models = sorted(list(set([g[2] for g in pr_df.groups])))
        palette = d3['Category10'][max(3, len(models))]
        colors = {model: palette[i] for i, model in enumerate(models)}

    tabs = []
    rank = 'global_rank'  # 'truth_sample_rank',
#         plot_row = []
#         for truth_sample in set([g[1] for g in pr_df.groups]):
#             hover = HoverTool(tooltips=[
#                     ("model", "@model"),
#                     ("bin", "@bin"),
#                     ("score (min, max)", "(@min_score, @max_score)"),
#                     ('n_alleles', '@n_alleles'),
#                     ('cum_alleles', '@cum_alleles'),
#                     ("data (x,y)", "($x, $y)")
#                 ],
#                 mode='vline'
#             )
    TOOLS = "hover,save,pan,box_zoom,reset,wheel_zoom".split(',')

    hover = HoverTool(
        tooltips=[
            ("model", "@model"),
            ("bin", "@bin"),
            ("score (min, max)", "(@min_score, @max_score)"),
            ('n_alleles', '@n_alleles'),
            ('cum_alleles', '@cum_alleles'),
            ('recall', '@recall'),
            ('precision', '@precision'),
            ('cum_fp', '@cum_fp'),
            ('cum_tp', '@cum_tp'),
            ('cum_fn', '@cum_fn'),
            ('f1', '@f1'),
            ('f2', '@f2'),
        ],
        # display a tooltip whenever the cursor is vertically in line with a glyph
        mode='vline'
    )
    p = figure(
        title=TRUTH_SAMPLE,
        x_axis_label='Recall',
        y_axis_label='Precision',
        tools=[hover] + [tool for tool in TOOLS if tool != 'hover'],
#         x_range=(0, 1.05), 
#         y_range=(0, 1)
    )
    p.xaxis[0].formatter = NumeralTickFormatter(format="0%")
    p.yaxis[0].formatter = NumeralTickFormatter(format="0.0%")

    circles = []
    for model in set([g[2] for g in pr_df.groups]):
        data = pr_df.get_group((rank, TRUTH_SAMPLE, model, snv)).copy()
        data['model'] = [model] * len(data)
        data['size'] = get_point_size_col(data['n_alleles'], size_prop)
        data['x_offset'] = data['recall'] + 0.025
        data['y_offset'] = data['precision']
        data['f2'] = [str("{:.2f}".format(t)) for t in data['f2']]
        source = ColumnDataSource(data)
        if bins_to_label is not None:
            label_data = data.copy()
            label_data = label_data.loc[label_data.bin.isin(bins_to_label)].copy()
            label_data = ColumnDataSource(label_data)
            p.circle(
                'recall',
                'precision',
                color=colors[model], 
                source=label_data
            )
            p.add_layout(
                LabelSet(
                    x='x_offset',
                    y='precision',
                    text='f2',
                    text_font_size='6pt',
                    x_offset=-10, 
                    y_offset=1,
                    text_color=colors[model],
                    source=label_data
                )
            )
        p.line(
            'recall',
            'precision',
            source=source,
            line_width=2,
        )
#             p.tools = [hover] + [tool for tool in TOOLS if tool != 'hover']
#     set_plots_defaults(p)
    return p
#         tabs.append(Panel(child=Row(children=plot_row), title=rank))
#     return Tabs(tabs=tabs)

# nbins // 20
p = plot_concordance_pr(df, snv=True, bins_to_label=range(0, nbins + 1, 1))
show(p)

In [151]:
df.head()

bin  min_score  max_score  \
rank_name         truth_sample model snv                                    
global_rank       NA12878      VQSR  False 0    0.0    17.9027    18.0431   
                                           1    1.0    17.8035    17.9027   
                                           2    2.0    17.6876    17.8035   
                                           3    3.0    17.5437    17.6876   
                                           4    4.0    17.3570    17.5436   
                                     True  102  0.0     6.6075     6.8471   
                                           103  1.0     6.5465     6.6075   
                                           104  2.0     6.4959     6.5465   
                                           105  3.0     6.4502     6.4959   
                                           106  4.0     6.4069     6.4502   
truth_sample_rank NA12878      VQSR  False 204  0.0    17.9023    18.0431   
                                           205  1.0    17.8026    17.9023   
                                           206  2.0    17.6861    17.8025   
                                           207  3.0    17.5413    17.6860   
                                           208  4.0    17.3530    17.5413   
                                     True  305  0.0     6.6075     6.8471   
                                           306  1.0     6.5464     6.6075   
                                           307  2.0     6.4958     6.5464   
                                           308  3.0     6.4501     6.4958   
                                           309  4.0     6.4068     6.4501   

                                                n_alleles     tp    fp  fn  \
rank_name         truth_sample model snv                                     
global_rank       NA12878      VQSR  False 0        10370   8881  1489   0   
                                           1        10370   8917  1453   0   
                                           2        10371   8952  1419   0   
                                           3        10370   8918  1452   0   
                                           4        10371   8945  1426   0   
                                     True  102      41026  37549  3477   0   
                                           103      41026  37506  3520   0   
                                           104      41026  37538  3488   0   
                                           105      41026  37469  3557   0   
                                           106      41026  37513  3513   0   
truth_sample_rank NA12878      VQSR  False 204      10411   8914  1497   0   
                                           205      10411   8958  1453   0   
                                           206      10412   8984  1428   0   
                                           207      10411   8958  1453   0   
                                           208      10412   8971  1441   0   
                                     True  305      41054  37574  3480   0   
                                           306      41055  37533  3522   0   
                                           307      41055  37563  3492   0   
                                           308      41055  37500  3555   0   
                                           309      41055  37531  3524   0   

                                                cum_alleles  cum_tp  cum_fp  \
rank_name         truth_sample model snv                                      
global_rank       NA12878      VQSR  False 0          10370    8881    1489   
                                           1          20740   17798    2942   
                                           2          31111   26750    4361   
                                           3          41481   35668    5813   
                                           4          51852   44613    7239   
                                     True  102        41026   37549    3477   
    

In [32]:
bins_to_label=range(0, nbins + 1, nbins // 20)
data = df.get_group(('truth_sample_rank', truth_sample, 'VQSR', True)).copy()
a = data.loc[data.bin.isin(bins_to_label)]
print(a)

NameError: name 'truth_sample' is not defined